In [0]:
%pip install geopandas==0.14.3 databricks-mosaic

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
from pyspark.sql.functions import *

In [0]:
from mosaic import enable_mosaic
enable_mosaic(spark, dbutils)


            DEPRECATION WARNING: 
                Please use a Databricks:
                    - Photon-enabled Runtime for performance benefits
                    - Runtime ML for spatial AI benefits
                Mosaic will stop working on this cluster after v0.3.x.


In [0]:
geoDF = (
  spark.read.format("json")
  .load("dbfs:/FileStore/map.geojson")
)
geoDF.show()

+--------------------+-----------------+
|            features|             type|
+--------------------+-----------------+
|[{{[[[20.45543129...|FeatureCollection|
+--------------------+-----------------+



In [0]:
geoDF = geoDF.select(explode("features").alias("feature"))

geoDF = geoDF.select(
    geoDF.feature.geometry.alias("raw_geometry")
)

geoDF.display()

raw_geometry
"List(List(List(List(20.455431296344386, 50.079930179734646), List(20.455431296344386, 49.89022550478154), List(20.784936593937516, 49.89022550478154), List(20.784936593937516, 50.079930179734646), List(20.455431296344386, 50.079930179734646))), Polygon)"


In [0]:
from mosaic import st_geomfromgeojson

geoDF = geoDF.withColumn("geometry", st_geomfromgeojson(to_json("raw_geometry")))

geoDF.display()

raw_geometry,geometry
"List(List(List(List(20.455431296344386, 50.079930179734646), List(20.455431296344386, 49.89022550478154), List(20.784936593937516, 49.89022550478154), List(20.784936593937516, 50.079930179734646), List(20.455431296344386, 50.079930179734646))), Polygon)","List(5, 4326, List(List(List(20.455431296344386, 50.079930179734646), List(20.455431296344386, 49.89022550478154), List(20.784936593937516, 49.89022550478154), List(20.784936593937516, 50.079930179734646), List(20.455431296344386, 50.079930179734646))), List(List()))"


In [0]:
from mosaic import st_point, st_area, st_distance, st_contains, st_intersects
pt = st_point(lit(21.238944752863773), lit(49.69250412428684))
olimpia = st_point(lit(20.621880800952454), lit(50.03471571202331))
          
(
  geoDF
  .withColumn("calculatedArea", abs(st_area("geometry")))
  .withColumn("calculatedLength", st_distance("geometry", pt))
  .withColumn("containsPoint", st_contains("geometry", olimpia))
  .withColumn("intersects", st_intersects(pt, olimpia))
  .select("geometry", "calculatedArea", "calculatedLength", "containsPoint", "intersects")
).show()

+--------------------+-------------------+-------------------+-------------+----------+
|            geometry|     calculatedArea|   calculatedLength|containsPoint|intersects|
+--------------------+-------------------+-------------------+-------------+----------+
|{5, 4326, [[[20.4...|0.06250869537523107|0.49519405557451796|         true|     false|
+--------------------+-------------------+-------------------+-------------+----------+

